[View in Colaboratory](https://colab.research.google.com/github/fromppf/keras/blob/keras-1/oxford_deepNLP_2017.ipynb)

### Practical 1: word2vec
Oxford CS - Deep NLP 2017
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/

[Yannis Assael, Brendan Shillingford, Chris Dyer]

This practical is presented as an IPython Notebook, with the code written for recent versions of Python 3. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press shift-enter. The return value of the last command will be displayed, if it is not None.

Potentially useful library documentation, references, and resources:

* IPython notebooks: https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction
* Numpy numerical array library: https://docs.scipy.org/doc/
* Gensim's word2vec: https://radimrehurek.com/gensim/models/word2vec.html
* Bokeh interactive plots: http://bokeh.pydata.org/en/latest/ (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka sklearn): http://scikit-learn.org/stable/documentation.html
* nltk NLP toolkit: http://www.nltk.org/
* tutorial for processing xml in python using lxml: http://lxml.de/tutorial.html (we did this for you below, but in case you need it in the future)

In [0]:
import numpy as np
import os
from random import shuffle
import re

In [3]:
!pip install bokeh

    100% |████████████████████████████████| 13.6MB 99kB/s 
  Running setup.py bdist_wheel for bokeh ... - \ | / - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/74/92/e2/b09e82290a5419035a97ee96b553fb94d95e6ae83641bd75c2
Successfully built bokeh


In [4]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

#### Part 0: Download the TED dataset

In [6]:
!pip install lxml

    100% |████████████████████████████████| 5.7MB 235kB/s 


In [0]:
import urllib.request
import zipfile
import lxml.etree

In [0]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [0]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

#### Part 1: Preprocessing
In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [10]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [0]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [12]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "<up to 20 characters>:", as shown in this example. Of course, this is an imperfect heuristic.

In [13]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [0]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [15]:
len(sentences_ted)

266694

In [16]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


#### Part 2: Word Frequencies
If you store the counts of the top 1000 words in a list called counts_ted_top1000, the code below will plot the histogram requested in the writeup.

In [0]:
import collections
count = collections.Counter()
for sentence in sentences_ted:
    for word in sentence:
        count[word] += 1
words_top_ted = [token_count_pair[0] for token_count_pair in count.most_common(1000)]
counts_ted_top1000 = [token_count_pair[1] for token_count_pair in count.most_common(1000)]

Plot distribution of top-1000 words

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

In [43]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

#### Part 3: Train Word2Vec

In [23]:
!pip install gensim

    100% |████████████████████████████████| 22.6MB 60kB/s 
    100% |████████████████████████████████| 133kB 7.7MB/s 
    100% |████████████████████████████████| 1.4MB 919kB/s 
    100% |████████████████████████████████| 4.2MB 319kB/s 
    100% |████████████████████████████████| 61kB 9.1MB/s 
    100% |████████████████████████████████| 552kB 2.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /content/.cache/pip/wheels/cf/54/36/b003d8c2d26aadffc21f0677009ed53cf9575a97fc71fbba76
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/31/9c/20/996d65ca104cbca940b1b053299b68459391c01c774d073126
Successfully built smart-open bz2file


In [0]:
from gensim.models import Word2Vec

In [0]:
model_ted = Word2Vec(sentences_ted, min_count=1)

#### Part 4: Ted Learnt Representations
Finding similar words: (see gensim docs for more functionality of most_similar)

In [26]:
model_ted.most_similar("man")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.8455406427383423),
 ('guy', 0.8122212886810303),
 ('girl', 0.7617380619049072),
 ('lady', 0.7529218196868896),
 ('boy', 0.7420065402984619),
 ('soldier', 0.735586404800415),
 ('gentleman', 0.7211829423904419),
 ('friend', 0.670264720916748),
 ('kid', 0.6646609306335449),
 ('poet', 0.6606656312942505)]

In [27]:
model_ted.most_similar("computer")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('machine', 0.7360857725143433),
 ('software', 0.7162219882011414),
 ('robot', 0.7097142338752747),
 ('3d', 0.6827031373977661),
 ('device', 0.6782822608947754),
 ('program', 0.6559253931045532),
 ('camera', 0.6505266427993774),
 ('interface', 0.6426190137863159),
 ('mechanical', 0.630643367767334),
 ('video', 0.6248986124992371)]

In [28]:
model_ted.most_similar("intelligence")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('communication', 0.762283444404602),
 ('knowledge', 0.756449282169342),
 ('awareness', 0.7544309496879578),
 ('computing', 0.7539945244789124),
 ('collective', 0.7391146421432495),
 ('engineering', 0.7336736917495728),
 ('development', 0.7313259840011597),
 ('resilience', 0.7281637787818909),
 ('civilization', 0.726922333240509),
 ('medicine', 0.725447952747345)]

In [31]:
model_ted.most_similar("deep")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('dark', 0.602084755897522),
 ('liquid', 0.587346076965332),
 ('ocean', 0.5783253312110901),
 ('sea', 0.5753763318061829),
 ('outer', 0.5650707483291626),
 ('atmosphere', 0.5639786720275879),
 ('frozen', 0.5588552355766296),
 ('flowing', 0.5413405299186707),
 ('fluid', 0.5388778448104858),
 ('landscape', 0.5358415246009827)]

In [32]:
model_ted.most_similar("learning")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('designing', 0.6522548794746399),
 ('understanding', 0.6049444079399109),
 ('teaching', 0.5891402959823608),
 ('knowledge', 0.586750328540802),
 ('discovering', 0.5859370827674866),
 ('sharing', 0.5825132131576538),
 ('programming', 0.5810202360153198),
 ('thinking', 0.575718879699707),
 ('learn', 0.5746056437492371),
 ('engaging', 0.5682476162910461)]

##### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable words_top_ted. The following code gets the corresponding vectors from the model, assuming it's called model_ted:

In [33]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [0]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [35]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

#### Part 5: Wiki Learnt Representations
Download dataset

In [0]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [0]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [0]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [40]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [0]:
model_wiki = Word2Vec(sentences_wiki, min_count=1)

In [0]:
count_wiki = collections.Counter()
for sentence in sentences_wiki:
    for word in sentence:
        count_wiki[word] += 1
words_top_wiki = [token_count_pair[0] for token_count_pair in count.most_common(1000)]

##### t-SNE visualization

In [48]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

In [49]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

NameError: ignored